In [1]:
from collections import defaultdict, Counter, OrderedDict
import numpy as np
from copy import copy
from itertools import groupby, permutations, product
import json
import matplotlib.pyplot as plt
import numpy as np
import math

# DATA

In [2]:
f = open("inputs/aoc11.txt").read()
f = f.splitlines()
f = [[c for c in l] for l in f]

In [12]:
test = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
"""
test_f = test.splitlines()
test_f = [[c for c in l] for l in test_f]
test_f_a = np.array(test_f)

# 1

In [3]:
def create_intervals(data):
    """
        Create a list of intervals out of set of ints.
    """
    if not data:
        return []
    data = sorted(data)
    intervals = []
    start = last = data[0]
    for i in data[1:]:
        diff = i - last
        if diff > 1:
            intervals.append((start, last))
            start = i
        last = i
    if start:        
        intervals.append((start, last))
    return intervals

In [54]:
class Universe:
    
    def __init__(self,f):
        
        self.f=f
        self.f_array = np.array(f)
        self.dots = np.array([[i=="." for i in line] for line in f])
        self.num_cols=self.dots.shape[0]
        self.num_rows=self.dots.shape[1]
        self.empty_cols = []
        self.empty_rows = []
        
        
        for i in np.arange(0,self.num_rows):
            if all(self.dots[i,:]):
                self.empty_rows.append(i)
        
        for i in np.arange(0,self.num_cols):
            if all(self.dots[:,i]):
                self.empty_cols.append(i)
        print("empty cols:",self.empty_cols)
        print("empty rows:",self.empty_rows)
    
    def expand_cols(self):
        
        new_list = []

        # expand cols
        all_indices = np.arange(0,self.num_cols)        
        ids_non_empty = set(all_indices)-set(self.empty_cols)
        non_empty_intervals = create_intervals(ids_non_empty)
        print("non-empty cols:",non_empty_intervals)
        lower_bounds = [min(s) for s in non_empty_intervals]
        indices = sorted(lower_bounds+self.empty_cols)
        print(indices)

        for col in indices:
            for s in non_empty_intervals:
                if col in s:
                    new_array = self.f_array[:,s[0]:s[1]+1]
            if col in u.empty_cols:
                new_array = np.hstack([self.f_array[:,col:col+1]]*2)
            new_list.append(new_array)
        self.f_array = np.concatenate(new_list,axis=1)   

    def expand_rows(self):
        
        new_list = []

        # expand cols
        all_indices = np.arange(0,self.num_rows)        
        ids_non_empty = set(all_indices)-set(self.empty_rows)
        non_empty_intervals = create_intervals(ids_non_empty)
        lower_bounds = [min(s) for s in non_empty_intervals]
        indices = sorted(lower_bounds+self.empty_rows)

        for row in indices:
            for s in non_empty_intervals:
                if row in s:
                    new_list.append(self.f_array[s[0]:s[1]+1,:])

            if row in u.empty_rows:
                new_array = np.vstack([self.f_array[row:row+1,:]]*2)
                new_list.append(new_array)

        self.f_array = np.concatenate(new_list,axis=0)  
        
    def get_galaxies(self):
        counter=0
        self.galaxies = {}
        for x in np.arange(0,self.f_array.shape[1]):
            for y in np.arange(0,self.f_array.shape[0]):
                if self.f_array[y,x]=="#":
                    counter=counter+1
                    self.galaxies[counter]=(x,y)
    
    def compute_shortest_path(self):
        self.distances = {}
        for g1,g1c in self.new_galaxies.items():
            for g2,g2c in self.new_galaxies.items():
                if ((g1c, g2c) not in self.distances) and  ((g2c, g1c) not in self.distances):
                    distance = abs(g1c[0]-g2c[0])+abs(g1c[1]-g2c[1])
                    self.distances[(g1c, g2c)]=distance
    
    def new_coords_galaxies(self, multiplier=2):
        self.new_x = []
        self.new_y = []
        
        num_empty_cols_before = 0
        for g_id, g_c in self.galaxies.items():
            
            xc = g_c[0]
            
            num_empty_cols_before = sum(np.array(self.empty_cols)<xc)
            print(g_id, g_c,  num_empty_cols_before)
            new_xc = xc + num_empty_cols_before*(multiplier-1)
            self.new_x.append(new_xc)
            
            yc = g_c[1]
            num_empty_rows_before = sum(np.array(self.empty_rows)<yc)
            new_yc = yc + num_empty_rows_before*(multiplier-1)
            self.new_y.append(new_yc)
        
        self.new_galaxies={}
        counter = 0
        for x,y in zip(self.new_x,self.new_y):
            counter=counter+1
            self.new_galaxies[counter]=(x,y)
        
        

In [60]:
u = Universe(f)
print(u.f_array)
#u.expand_cols()
#u.expand_rows()
print(u.f_array.shape)

u.get_galaxies()
u.new_coords_galaxies(multiplier=1e6)
u.compute_shortest_path()
sum(u.distances.values())

empty cols: [12, 13, 44, 56, 81, 82, 84, 103]
empty rows: [57, 59, 82, 83, 87, 89]
[['.' '.' '#' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ...
 ['.' '.' '.' ... '.' '#' '.']
 ['.' '.' '#' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']]
(140, 140)
1 (0, 58) 0
2 (1, 24) 0
3 (1, 104) 0
4 (2, 0) 0
5 (2, 18) 0
6 (2, 40) 0
7 (2, 68) 0
8 (2, 90) 0
9 (2, 131) 0
10 (2, 138) 0
11 (3, 32) 0
12 (3, 99) 0
13 (3, 109) 0
14 (3, 122) 0
15 (4, 10) 0
16 (4, 81) 0
17 (4, 94) 0
18 (4, 114) 0
19 (5, 28) 0
20 (5, 47) 0
21 (5, 128) 0
22 (6, 20) 0
23 (6, 54) 0
24 (6, 74) 0
25 (7, 4) 0
26 (7, 134) 0
27 (8, 60) 0
28 (8, 121) 0
29 (9, 34) 0
30 (9, 99) 0
31 (10, 1) 0
32 (10, 14) 0
33 (10, 39) 0
34 (10, 45) 0
35 (10, 69) 0
36 (10, 115) 0
37 (10, 138) 0
38 (11, 7) 0
39 (11, 30) 0
40 (11, 52) 0
41 (11, 80) 0
42 (11, 90) 0
43 (11, 107) 0
44 (14, 18) 2
45 (14, 36) 2
46 (14, 56) 2
47 (14, 131) 2
48 (15, 4) 2
49 (15, 26) 2
50 (15, 86) 2
51 (15, 122) 2
52 (16, 43) 2
53 (16, 63) 2
54

611998089572.0

In [59]:
u.new_galaxies

{1: (0.0, 2.0),
 2: (0.0, 2000007.0),
 3: (1.0, 1000004.0),
 4: (1000002.0, 0.0),
 5: (1000003.0, 2000007.0),
 6: (2000004.0, 1000003.0),
 7: (2000005.0, 1.0),
 8: (2000005.0, 2000006.0),
 9: (3000006.0, 1000005.0)}

In [48]:
u.expand_cols()
u.f_array

non-empty cols: [(0, 1), (3, 4), (6, 7), (9, 9)]
[0, 2, 3, 5, 6, 8, 9]


array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

In [49]:
u.f_array.shape

(10, 13)